In [2]:
import glob
import pandas as pd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from cv2 import resize, imread
from pydub import AudioSegment

C:\Users\USER\Anaconda3\envs\MachineLearningEnv\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


**ENCODING LABEL**

In [3]:
def classEncoder(start,end,crackle,wheeze,idx):
    #status label = [exhale,inhale] 
    if idx%2==0:
        status = 1
    else:
        status = 0
    #class label = [normal,crackle,wheeze,crackle&wheeze]
    if wheeze=="1" and crackle=="1":
        classes = "3"
    else:
        if wheeze=="1" and crackle=="0":
            classes = "2"
        else:
            if wheeze=="0" and crackle=="1":
                classes = "1"
            else:
                classes = "0"
    return [start,end,str(status),classes]

In [4]:
files = glob.glob("Data/Audio/*.txt")
lines = []
idAud = 0
for fileName in files:
    file = open("Data/Audio/"+fileName[(fileName.index("\\"))+1:])
    i = 0
    for line in file:
        line = line.split("\t")
        line = classEncoder(line[0],line[1],line[2],line[3][0],i)
        line.insert(0,fileName[(fileName.index("\\"))+1:-4])
        line.insert(0,str(idAud))
        lines.append(line)
        idAud+=1
        i+=1
    file.close()
lungSound = pd.DataFrame(lines,columns=["Id","File","StartRC","EndRC","Status","Class"])
lungSound.head(20)

,Id,File,StartRC,EndRC,Status,Class
0,0,101,0.036,1.907,1,0
1,1,101,1.907,4.521,0,0
2,2,101,4.521,7.193,1,0
3,3,101,7.193,9.75,0,0
4,4,101,9.75,12.407,1,0
5,5,101,12.407,15.079,0,0
6,6,101,15.079,17.521,1,0
7,7,101,17.521,19.95,0,0
8,8,102,0.107,2.207,1,0
9,9,102,2.207,4.064,0,1


In [5]:
lungSound.to_csv(r'Data/Preprocessed Data/LabelEncoded.csv',index=False)

**SPLITTING AUDIO**

In [6]:
done = glob.glob("Data/Preprocessed Data/Audio/*.wav")

for i in range(lungSound.shape[0]):    
    if "Data/Preprocessed Data/Audio\\"+str(i)+".wav" not in done:
        sub = lungSound[lungSound["Id"]==str(i)]
        filename = "Data/Audio/",str(sub["File"].values[0]),".wav"
        filename = "".join(filename)
        a = AudioSegment.from_file(filename) 
        first_second = a[:1000] # get the first second of an mp3 
        startRC = float(sub["StartRC"].values[0])*1000
        endRC = float(sub["EndRC"].values[0])*1000
        slice = a[startRC:endRC] # get a slice from 5 to 10 seconds of an mp3
        slice.export("Data/Preprocessed Data/Audio/"+str(i)+".wav", format="wav")

**RESIZING SPECTOGRAM**

In [7]:
files = glob.glob("Data/Spectogram/*.png")
dones = glob.glob("Data/Preprocessed Data/Spectogram/*.png")

#Mengetahui gambar yang belum di-resize
for resized in dones:
    if resized:
        files.remove("Data/Spectogram\\"+resized[(resized.index("\\"))+1:])

for fileName in files:
    plotpath = "Data/Preprocessed Data/Spectogram/"+fileName[(fileName.index("\\"))+1:-4]
    img = mpimg.imread("Data/Spectogram/"+fileName[(fileName.index("\\"))+1:])
    img_rescale = resize(img,(800,600))
    plt.axis("off")
    plt.imshow(img_rescale)
    plt.savefig(plotpath, bbox_inches="tight")

**MAKE CSV FOR MODEL**

In [14]:
def append_ext(id):
    return id+".png"

preprocessed_df = pd.read_csv('Data/Preprocessed Data/LabelEncoded.csv')
df_model = preprocessed_df[['Id','Class']].astype("str")
df_model["Id"]=df_model["Id"].apply(append_ext)
df_model.head()

,Id,Class
0,0.png,0
1,1.png,0
2,2.png,0
3,3.png,0
4,4.png,0


In [15]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2369 entries, 0 to 2368
Data columns (total 2 columns):
Id       2369 non-null object
Class    2369 non-null object
dtypes: object(2)
memory usage: 37.1+ KB


In [16]:
df_model['Class'] = df_model['Class'].astype('int')
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2369 entries, 0 to 2368
Data columns (total 2 columns):
Id       2369 non-null object
Class    2369 non-null int32
dtypes: int32(1), object(1)
memory usage: 27.8+ KB


In [17]:
df_model.to_csv(r'Data/Preprocessed Data/DataModel.csv',index=False)